# PyTorch
In this exercise, we will look at some basic functionality of PyTorch. Your are free to use other DL frameworks for your exercises and your project. However, the master solutions and code examples will be in PyTorch.

The [PyTorch documentation](https://pytorch.org/docs/stable/index.html) offers information on its functionality. A lot of the time, your specific question will also have been asked on the [PyTorch Forum](https://discuss.pytorch.org/), often with competent answers by the core developers (Google will find the relevant thread for you).

First, we have to install PyTorch. We will install the basic version for this exercise. For your project, if you want to run on a GPU, you'll have to make sure to have a PyTorch version installed that is compatible with the CUDA version of your NVIDIA drivers. PyTorch has an [installation guide](https://pytorch.org/get-started/locally/) that will help you with getting the right version.

In [1]:
%pip install -q -U numpy
%pip install -q torch ipywidgets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.1.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import numpy as np

## Tensor operations
Most of PyTorch's operations have the same name as in NumPy. The basic object for storing data is the `torch.tensor`, the equivalent of the `np.array`. With the help of the [Tensor tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html), do the following:

- Create a `torch.tensor` with the elements `[[1, 2], [3, 4]]`
- Create an `np.array` with the same elements
- Convert from one to the other
- Check how the values of one change when you change the other
- Create a tensor of ones/zeros with the same shape and dtype
- Create a random tensor of the same shape
- Print the tensor's shape, data type and device
- Try to move it to the GPU
- Check out indexing/slicing operations, and how you can assign values to a slice.
- Combine tensors with `torch.cat` and `torch.stack`. What are the differences?
- Multiply tensors, element-wise and with matrix multiplication.

In [3]:
base = [[1, 2], [3, 4]]
tensor = torch.tensor(base)
ndarr = np.array(base)
print(tensor)
print(ndarr)

print(tensor.numpy())
print(torch.from_numpy(ndarr))

tensor[0,0] = 5
print(tensor)
print(ndarr)
ndarr[0,0] = 6
print(tensor)
print(ndarr)

print(torch.zeros(tensor.shape))
print(torch.ones(tensor.shape))
print(torch.rand(tensor.shape))

print(tensor.shape, tensor.dtype, tensor.device)

if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

tensor[0] = 8
print(tensor)
tensor[:, 0] = 9
print(tensor)

print(torch.cat([tensor, tensor]))
print(torch.stack([tensor, tensor]))

print(tensor * tensor)
tensor = tensor.to("cpu")
print(tensor @ tensor)

tensor([[1, 2],
        [3, 4]])
[[1 2]
 [3 4]]
[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]])
tensor([[5, 2],
        [3, 4]])
[[1 2]
 [3 4]]
tensor([[5, 2],
        [3, 4]])
[[6 2]
 [3 4]]
tensor([[0., 0.],
        [0., 0.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[0.8505, 0.1239],
        [0.3334, 0.0039]])
torch.Size([2, 2]) torch.int64 cpu
Device tensor is stored on: cuda:0
tensor([[8, 8],
        [3, 4]], device='cuda:0')
tensor([[9, 8],
        [9, 4]], device='cuda:0')
tensor([[9, 8],
        [9, 4],
        [9, 8],
        [9, 4]], device='cuda:0')
tensor([[[9, 8],
         [9, 4]],

        [[9, 8],
         [9, 4]]], device='cuda:0')
tensor([[81, 64],
        [81, 16]], device='cuda:0')
tensor([[153, 104],
        [117,  88]])


## Neural Network Basics
Solve the followings tasks with the help of the [Neural networks tutorial](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html).

The `nn.Module` is the basic class for layers, networks and models. All parameters of an `nn.Module` are automatically discovered by PyTorch and updated by back-propagation.

First, define a neural network (as a subclass of `nn.Module`) with two linear layers and a ReLU non-linearity in between. Make the input, output, and inner dimensions parameters of your network.

In [4]:
import torch.nn as nn

In [5]:
class Network(nn.Module):
    def __init__(self, input_dim=16, output_dim=16, hidden_dim=16):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, data):
        output = self.fc1(data)
        output = nn.functional.relu(output)
        output = self.fc2(output)
        return output
net = Network()

Print the parameters of your network.

In [6]:
print(net)

Network(
  (fc1): Linear(in_features=16, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
)


Run a single forward-pass with a random input.

In [7]:
out = net(torch.rand(1,16))

Define a `nn.MSELoss` and a random target.

In [8]:
criterion = nn.MSELoss()
target = torch.rand(1, 16)

Compute the loss and run backpropagation.

In [9]:
loss = criterion(out, target)
loss

tensor(0.3040, grad_fn=<MseLossBackward0>)

Update the parameters of your network with a learning rate of 0.01.

In [10]:
net.zero_grad()
loss.backward()
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

Use the `AdamOptimizer` instead to update your parameters (see the [torch.optim documentation](https://pytorch.org/docs/stable/optim.html)).

In [11]:
import torch.optim as optim

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(torch.rand(1,16))
loss = criterion(output, target)
loss.backward()
optimizer.step()